In [ ]:
!pip --version

# To install pytorch -------------------------------------------------------------------------------------
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# To install Tensorflow ----------------------------------------------------------------------------------
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# To install Flax ----------------------------------------------------------------------------------------
!pip install flax

# or to install the latest version of Flax:
!pip install --upgrade git+https://github.com/google/flax.git

# Set up transformers
!pip install git+https://github.com/huggingface/transformers

# Check if installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

!pip install transformers[torch]

!pip install accelerate -U

!pip install -q datasets peft evaluate

In [14]:
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer )
from peft import ( get_peft_config, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, PeftType,PromptEncoderConfig,PeftModelForSequenceClassification)
from peft import PromptEmbedding, PromptTuningConfig
from datasets import load_dataset
import evaluate
from transformers import AdamW
import torch
import numpy as np
from peft import LoraModel, LoraConfig
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


dataset = load_dataset("sst2")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2) # as output 0 or 1
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", padding_side = "right")
model = model.to('cuda')
# optimizer = AdamW(model.parameters(), lr= 0.005, eps = 1e-8)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
!pip install loralib

  Obtaining dependency information for loralib from https://files.pythonhosted.org/packages/f1/e7/a4362bf791bca17d2d91e7c69483185ab03d5aa05dd10391eff2e179a685/loralib-0.1.2-py3-none-any.whl.metadata


In [12]:
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2023.12.2 which is incompatible.


In [2]:
for name, modules in model.named_modules():
  print(name)


bert
bert.embeddings
bert.embeddings.word_embeddings
bert.embeddings.position_embeddings
bert.embeddings.token_type_embeddings
bert.embeddings.LayerNorm
bert.embeddings.dropout
bert.encoder
bert.encoder.layer
bert.encoder.layer.0
bert.encoder.layer.0.attention
bert.encoder.layer.0.attention.self
bert.encoder.layer.0.attention.self.query
bert.encoder.layer.0.attention.self.key
bert.encoder.layer.0.attention.self.value
bert.encoder.layer.0.attention.self.dropout
bert.encoder.layer.0.attention.output
bert.encoder.layer.0.attention.output.dense
bert.encoder.layer.0.attention.output.LayerNorm
bert.encoder.layer.0.attention.output.dropout
bert.encoder.layer.0.intermediate
bert.encoder.layer.0.intermediate.dense
bert.encoder.layer.0.intermediate.intermediate_act_fn
bert.encoder.layer.0.output
bert.encoder.layer.0.output.dense
bert.encoder.layer.0.output.LayerNorm
bert.encoder.layer.0.output.dropout
bert.encoder.layer.1
bert.encoder.layer.1.attention
bert.encoder.layer.1.attention.self
bert.e

In [15]:
config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.01,
)

model = LoraModel(model, config, "default")

total_params = 0
trainable_params = 0

# trainable_layers = [model.prompt_encoder, model.classifier]
for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for p in model.classifier.parameters():
    p.requires_grad = True
        
optimizer = AdamW(model.parameters(), lr= 0.005, eps = 1e-8)

print("total:",total_params)
print("trainable_param",trainable_params)

total: 4394370
trainable_param 0


C:\Users\shantanu\anaconda3\envs\myenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
# Preprocessing
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Assuming the dataset has a 'train' split, modify this according to your dataset's splits
data_split = dataset['train']

# Convert the dataset split to a pandas DataFrame for easier splitting
df = data_split.to_pandas()

# Split the dataset into train and test sets using train_test_split from sklearn
# df1, df2 = train_test_split(df, test_size = 0.5, random_state = 42)
train_df, test_df = train_test_split(df , test_size=0.2, random_state=42)

# Convert the splits back to datasets
train_dataset = train_df.reset_index(drop=True)
test_dataset = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

x_train = list(train_dataset["sentence"])
y_train = list(train_dataset["label"])

x_test = list(test_dataset["sentence"])
y_test = list(test_dataset["label"])

X_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512)
# X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test, padding=True, truncation = True, max_length = 512)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
# val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 1024, drop_last = True)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = 1024, drop_last = True)


C:\Users\shantanu\anaconda3\envs\myenv\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [ ]:
def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        #batch = tuple(t.to("cuda") for t in batch)

        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs[:2]

            log = logits.cpu()
            log = log.numpy()

            preds = np.argmax(log, axis=1)
            labels = batch['labels'].cpu().numpy()

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

# Training loop
optimizer.zero_grad()  # Explicitly zero the gradient buffers

for epoch in range(60):  # Number of epochs
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # print(f"input_ids size: {input_ids.size()}")
        # print(f"attention_mask size: {attention_mask.size()}")
        # print(f"labels size: {labels.size()}")

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # print(outputs)
        loss = outputs[0]
        # print(loss)
        loss.backward()
        optimizer.step()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim = -1)
        #metric.add_batch(predictions = predictions, references = batch["labels"])

    eval_loss, eval_accuracy = evaluate(model)
    print("eval loss",eval_loss)
    print("accuracy: ",eval_accuracy)
    print("ends")
    #metric.compute()
    #print(metric)

    # Validation
    model.eval()
    for batch in test_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            # Further validation steps if needed

    model.train()  # Set the model back to training mode


eval loss 0.6226080197554368
accuracy:  0.6748046875
ends
eval loss 0.6026518207329971
accuracy:  0.6843449519230769
ends
eval loss 0.596383484510275
accuracy:  0.6915564903846154
ends
eval loss 0.5922278945262616
accuracy:  0.6917818509615384
ends
eval loss 0.5907220198557928
accuracy:  0.6949368990384616
ends
eval loss 0.5891422400107751
accuracy:  0.6934344951923077
ends
eval loss 0.5891852516394395
accuracy:  0.6967397836538461
ends
eval loss 0.5867767471533555
accuracy:  0.6940354567307693
ends
eval loss 0.587697079548469
accuracy:  0.6881760817307693
ends
eval loss 0.587439802976755
accuracy:  0.6938100961538461
ends
eval loss 0.5866657403799204
accuracy:  0.6887770432692307
ends
eval loss 0.5864757253573492
accuracy:  0.6911057692307693
ends
eval loss 0.585871146275447
accuracy:  0.6962139423076923
ends
eval loss 0.5859301319489112
accuracy:  0.6946364182692307
ends
eval loss 0.586718696814317
accuracy:  0.6935847355769231
ends
eval loss 0.5855791843854464
accuracy:  0.692983774